In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List
%matplotlib inline

In [2]:
# step 1 - read in the files
dimensions_matrix = pd.read_excel('./test_data/Dimensions_Matrix.xlsx')
cutoff_matrix = pd.read_excel('./test_data/cutoff_data.xlsx')
weigths_matrix = pd.read_excel('./test_data/Weights.xlsx')

In [5]:
def get_deprivation_matrix(dimensions_matrix: pd.DataFrame, cutoff_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the deprivation matrix (g0) based on dimensions and cutoff values.

    Args:
        dimensions_matrix (pd.DataFrame): DataFrame containing dimensions data.
        cutoff_matrix (pd.DataFrame): DataFrame containing cutoff values.

    Returns:
        pd.DataFrame: Deprivation matrix (g0).
    """
    # Get column names for the dataframes
    dimensions_column_names: List[str] = dimensions_matrix.columns

    # Create a copy of the dimensions matrix as the deprivation matrix
    deprivation_matrix: pd.DataFrame = dimensions_matrix.copy()

    # Generate values for the deprivation matrix based on dimensions and cutoff values
    for index, col_name in enumerate(dimensions_column_names):
        cutoff_value = cutoff_matrix.iloc[index]['Cutoff']
        deprivation_matrix[col_name] = deprivation_matrix[col_name].apply(lambda value: 1 if value == cutoff_value else 0)

    return deprivation_matrix

In [9]:
def calculate_weighted_deprivation_matrix(deprivation_matrix: pd.DataFrame, weights_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the weighted deprivation matrix based on the deprivation matrix and weights.

    Args:
        deprivation_matrix (pd.DataFrame): Deprivation matrix (g0).
        weights_matrix (pd.DataFrame): DataFrame containing weights.

    Returns:
        pd.DataFrame: Weighted deprivation matrix.
    """
    # Iterate through columns of the deprivation matrix
    for col_name in deprivation_matrix.columns:
        # Get the weight corresponding to the column
        weight = weights_matrix.loc[weights_matrix.index == col_name, 'Weight'].values[0]
        
        # Multiply the values in the column by the weight
        deprivation_matrix[col_name] = deprivation_matrix[col_name] * weight
    
    return deprivation_matrix

In [11]:
def calculate_deprevation_scores(weighted_deprivation_matrix : pd.DataFrame) -> pd.DataFrame:
    number_of_rows = len(weighted_deprivation_matrix.columns)
    data = { 'Scores' : [0] * number_of_rows}
    deprevation_scores = pd.DataFrame(data)
    for index in range(number_of_rows):
        deprevation_score = weighted_deprivation_matrix.iloc[index].sum()
        deprevation_scores.iloc[index, 'Scores'] = deprevation_score
    return deprevation_scores
        

SyntaxError: invalid syntax (1078661237.py, line 2)